In [1]:
import scala.util.matching
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.functions.{min, max, mean, stddev, typedLit, when}
import org.apache.spark.sql.types.{IntegerType, DoubleType}

println("all good")

all good


data is in the format:

{
"conversionSymbol": "",
"volumeto": 6685400.72, 
"high": 9165.89, 
"low": 9144.6, 
"time": 1595030400, 
"volumefrom": 730.39, 
"close": 9151.68, 
"open": 9156.79, 
"conversionType": "direct"
}


In [12]:
case class TradingInfo (time: Int,
                        high: Double,
                        low: Double,
                        open: Double,
                        volumefrom: Double,
                        volumeto: Double, 
                        close: Double,
                        conversionType: String,
                        conversionSymbol: String                        
                       )

def createTradingInfo(data: String): TradingInfo = {
    val s = data.split(", ")
    return TradingInfo(s(0).split(": ")(1).toInt, s(1).split(": ")(1).toDouble, s(2).split(": ")(1).toDouble, 
                       s(3).split(": ")(1).toDouble, s(4).split(": ")(1).toDouble, s(5).split(": ")(1).toDouble, 
                       s(6).split(": ")(1).toDouble, s(7).split(": ")(1), s(8).split(": ")(1))
}

// val filename = "../Datasets/bitcoin_json.txt"
val filename = "../Datasets/bitcoin_data_max.txt"
val tradingInformation = sc.textFile(filename).
            flatMap(l => l.split("},")).
            map(x => x.replace("[", "")).
            map(x => x.replace("{", "")).
            map(x => x.replace("\"", "")).
            map(createTradingInfo)
            .cache()

tradingInformation.take(3).foreach(println)

println("All good")

TradingInfo(1595574000,9516.47,9477.19,9484.19,1619.05,1.537644208E7,9516.47,\direct\,\\)
TradingInfo(1595577600,9518.18,9494.68,9516.47,1147.74,1.09079891E7,9514.87,\direct\,\\)
TradingInfo(1595581200,9536.29,9514.4,9514.87,1017.91,9698013.33,9536.26,\direct\,\\)
All good


defined class TradingInfo
filename = ../Datasets/bitcoin_data_max.txt
tradingInformation = MapPartitionsRDD[193] at map at <console>:47


createTradingInfo: (data: String)TradingInfo


MapPartitionsRDD[193] at map at <console>:47

In [13]:
// Data Action

val df = spark.createDataFrame(tradingInformation).sort("time")
val df1 = df.withColumn("midPrice",($"high" + $"low")/2)

// df1.show(5)

val ratio: Double = 0.9
val training_size = df1.count*ratio
val testing_size = df1.count - 10

val training_position = df1.take(training_size.toInt).last.getInt(0)
val testing_position = df1.take(testing_size.toInt).last.getInt(0)

// val df2 = df1.withColumn("dataset", when(col("time") <= training_position,"training")
//       .when(col("time") > training_position and col("time") <= testing_position,"validation")
//       .otherwise("testing"))
// df2.show(5)


val training_set = df1.filter($"time" <= training_position).cache()
val validation_set = df1.filter($"time" > training_position).filter($"time" <= testing_position).cache()
val testing_set = df1.filter($"time" > testing_position).cache()


// training_set.show(5)
// validation_set.show(5)
// testing_set.show(5)


// println(testing_set.count)

println("All good")

All good


df = [time: int, high: double ... 7 more fields]
df1 = [time: int, high: double ... 8 more fields]
ratio = 0.9
training_size = 12602.7
testing_size = 13993
training_position = 1597726800
testing_position = 1602734400
training_set = [time: int, high: double ... 8 more fields]
validation_set = [time: int, high: double ... 8 more fields]
testing_set = [time: int, high: double ... 8 more fields]


[time: int, high: double ... 8 more fields]

In [29]:
// training_set
// validation_set
// testing_set
val data_type = "cleaned_data_socialMedia/training_set"
val dataset = training_set

val statistics = dataset.agg(mean("midPrice"), stddev("midPrice"), min("midPrice"), max("midPrice"),
                                  mean("volumefrom"), stddev("volumefrom"), min("volumefrom"), max("volumefrom"),
                                  mean("volumeto"), stddev("volumeto"), min("volumeto"), max("volumeto")).head()

val midPrice_mean = statistics.getDouble(0)
val midPrice_std = statistics.getDouble(1)
val midPrice_min = statistics.getDouble(2)
val midPrice_max = statistics.getDouble(3)
val midPrice_range = midPrice_max - midPrice_min

// // VolumeFrom
val volumefrom_mean = statistics.getDouble(4)
val volumefrom_std = statistics.getDouble(5)
val volumefrom_min = statistics.getDouble(6)
val volumefrom_max = statistics.getDouble(7)
val volumefrom_range = volumefrom_max - volumefrom_min

// // VolumeTo
val volumeto_mean = statistics.getDouble(8)
val volumeto_std = statistics.getDouble(9)
val volumeto_min = statistics.getDouble(10)
val volumeto_max = statistics.getDouble(11)
val volumeto_range = volumeto_max - volumeto_min

// // // from -1 to 1 
// // //val new_dataset = dataset.withColumn("midPriceNorm",($"midPrice" - midPrice_mean)/midPrice_std)
// // // from 0 to 1 
val new_dataset = dataset.withColumn("midPriceNorm",(($"midPrice" - midPrice_min)/midPrice_range).cast(DoubleType))
               .withColumn("volumefromNorm",(($"volumefrom" - volumefrom_min)/volumefrom_range).cast(DoubleType))
               .withColumn("volumetoNorm",(($"volumeto" - volumeto_min)/volumeto_range).cast(DoubleType))
               .cache()
               .as("df_2")

new_dataset.show(5)
new_dataset.printSchema()

println("All good")

+----------+-------+-------+-------+----------+-------------+-------+--------------+----------------+------------------+--------------------+--------------------+--------------------+
|      time|   high|    low|   open|volumefrom|     volumeto|  close|conversionType|conversionSymbol|          midPrice|        midPriceNorm|      volumefromNorm|        volumetoNorm|
+----------+-------+-------+-------+----------+-------------+-------+--------------+----------------+------------------+--------------------+--------------------+--------------------+
|1552370400|3847.36|3841.99|3843.46|   1150.58|   4412932.61|3846.66|      \direct\|              \\|          3844.675|                 0.0|0.007802768998301...|0.005782282635044576|
|1552374000| 3852.5|3846.16|3846.66|    1551.8|   5984981.18|3850.52|      \direct\|              \\|           3849.33|4.730469917056464...|0.010524369849756021| 0.00784282947856672|
|1552377600|3886.31|3849.94|3850.52|   3193.81|1.238809879E7|3884.04|      \dire

data_type = cleaned_data_socialMedia/training_set
dataset = [time: int, high: double ... 8 more fields]
statistics = [8503.246020472938,1906.3696808766083,3844.675,13685.135,2024.5121378511346,2739.3449201142676,0.2912,147420.88,1.713158637164259E7,2.1539473974606913E7,1468.16,7.6292931663E8]
midPrice_mean = 8503.246020472938
midPrice_std = 1906.3696808766083
midPrice_min = 3844.675
midPrice_max = 13685.135
midPrice_range = 9840.46
volumefrom_mean = 2024.5121378511346
volumefrom_std = 2739.3449201142676
volumefrom_min = 0.2912
volumefrom_max = 147420.88
volumefrom_range = 147420.5888
volumeto_mean = 1.7131586371...


lastException: Throwable = null


1.7131586371...

In [ ]:
// Saving data as Text for complete bitcoin timeframe without socialData
// save it 3 times, one for each data set (training / validation / testing )
// don't forget to delete the previous folder 
val pre_processed_dataset = new_dataset.withColumn("empty", typedLit(Seq(1)))
pre_processed_dataset.rdd.repartition(1).saveAsTextFile(data_type)

println("All good")

In [30]:
// Social data is a colelction of various values from different social platforms 
// Explanation of the values can be found at: https://min-api.cryptocompare.com/documentation?key=Social&cat=historicalHourSocialStats
case class SocialData (time: Int,
                        comments: Int,
                        posts: Int,
                        followers: Int,
                        points: Int,
                        overview_page_views: Int,
                        analysis_page_views: Int,
                        markets_page_views: Int,
                        charts_page_views: Int,
                        trades_page_views: Int,
                        forum_page_views: Int,
                        influence_page_views: Int,
                        total_page_views: Int,
                        fb_likes: Int,
                        fb_talking_about: Int,
                        twitter_followers: Int,
                        twitter_following: Int,
                        twitter_lists: Int,
                        twitter_favourites: Int,
                        twitter_statuses: Int,
                        reddit_subscribers: Int,
                        reddit_active_users: Int,
                        reddit_posts_per_hour: Double,
                        reddit_posts_per_day: Double,
                        reddit_comments_per_hour: Double,
                        reddit_comments_per_day: Double,
                        code_repo_stars: Int,
                        code_repo_forks: Int,
                        code_repo_subscribers: Int,
                        code_repo_open_pull_issues: Int,
                        code_repo_closed_pull_issues: Int,
                        code_repo_open_issues: Int,
                        code_repo_closed_issues: Int,
                        code_repo_contributors: Int                     
                       )

def createSocialData(data: String): SocialData = {
    val s = data.split(", ")
    return SocialData(extractAndConvertToInt(s(0)),    extractAndConvertToInt(s(1)),     extractAndConvertToInt(s(2)),    extractAndConvertToInt(s(3)),
                      extractAndConvertToInt(s(4)),    extractAndConvertToInt(s(5)),     extractAndConvertToInt(s(6)),    extractAndConvertToInt(s(7)),
                      extractAndConvertToInt(s(8)),    extractAndConvertToInt(s(9)),     extractAndConvertToInt(s(10)),   extractAndConvertToInt(s(11)),
                      extractAndConvertToInt(s(12)),   extractAndConvertToInt(s(13)),    extractAndConvertToInt(s(14)),   extractAndConvertToInt(s(15)),
                      extractAndConvertToInt(s(16)),   extractAndConvertToInt(s(17)),    extractAndConvertToInt(s(18)),   extractAndConvertToInt(s(19)),
                      extractAndConvertToInt(s(20)),   extractAndConvertToInt(s(21)),    extractAndConvertToDouble(s(22)),extractAndConvertToDouble(s(23)),
                      extractAndConvertToDouble(s(24)),extractAndConvertToDouble(s(25)), extractAndConvertToInt(s(26)),   extractAndConvertToInt(s(27)),
                      extractAndConvertToInt(s(28)),   extractAndConvertToInt(s(29)),    extractAndConvertToInt(s(30)),   extractAndConvertToInt(s(31)),
                      extractAndConvertToInt(s(32)),   extractAndConvertToInt(s(33)))
}

def extractAndConvertToInt(attribute: String): Int = {
    return attribute.split(": ")(1).toInt
}

def extractAndConvertToDouble(attribute: String): Double = {
    return attribute.split(": ")(1).toDouble
}


val filename = "../Datasets/bitcoin_social_max.txt"
val socialDataInformation = sc.textFile(filename)
            .flatMap(l => l.split("},"))
            //.map(x => x.replace("\\", ""))
            .map(x => x.replace("[", ""))
            .map(x => x.replace("{", ""))
            .map(x => x.replace("\"", ""))
            .map(x => x.replace("]", "")) // Why do i need this here and not in the previous parser?
            .map(x => x.replace("}", "")) // -||-
            .map(createSocialData)
            .cache()

            

socialDataInformation.take(1).foreach(println)

println("All good")

SocialData(1595574000,356082,120973,78756,8554045,30012388,1094231,1600257,9444371,800616,9097523,62746,52112132,39654,26,844049,165,6631,1000,20316,1523948,3900,4.32,103.63,60.44,1450.64,44258,26102,3500,387,13297,543,5043,891)
All good


defined class SocialData
filename = ../Datasets/bitcoin_social_max.txt
socialDataInformation = MapPartitionsRDD[477] at map at <console>:92


createSocialData: (data: String)SocialData
extractAndConvertToInt: (attribute: String)Int
extractAndConvertToDouble: (attribute: String)Double


MapPartitionsRDD[477] at map at <console>:92

In [31]:
// Merge TradingInfo with SocialData

val socialDataDf = spark.createDataFrame(socialDataInformation).sort("time")
val mergedDataDf = new_dataset.join(socialDataDf, Seq("time")).cache()
mergedDataDf.show(1)

+----------+-------+-------+-------+----------+----------+-------+--------------+----------------+--------+------------+--------------------+--------------------+--------+-----+---------+-------+-------------------+-------------------+------------------+-----------------+-----------------+----------------+--------------------+----------------+--------+----------------+-----------------+-----------------+-------------+------------------+----------------+------------------+-------------------+---------------------+--------------------+------------------------+-----------------------+---------------+---------------+---------------------+--------------------------+----------------------------+---------------------+-----------------------+----------------------+
|      time|   high|    low|   open|volumefrom|  volumeto|  close|conversionType|conversionSymbol|midPrice|midPriceNorm|      volumefromNorm|        volumetoNorm|comments|posts|followers| points|overview_page_views|analysis_page_vie

socialDataDf = [time: int, comments: int ... 32 more fields]
mergedDataDf = [time: int, high: double ... 44 more fields]


[time: int, high: double ... 44 more fields]

In [32]:
// Saving data as Text

// don't forget to delete the previous folder 
val finalDf = mergedDataDf.withColumn("empty", typedLit(Seq(1)))
finalDf.rdd.repartition(1).saveAsTextFile(data_type)

println("All good")


All good


finalDf = [time: int, high: double ... 45 more fields]


[time: int, high: double ... 45 more fields]